In [15]:
#Tratando as colunas numéricas com base nos clusters provenientes das colunas categóricas e utilizando o KMeans após o tratamento
#Importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go

In [16]:
#Carregando o dataset já com informações sobre os clusters categóricos
dataset = pd.read_parquet('dataset_clusters_generos_paises_linguas.parquet')

In [17]:
#Escalando os valores para evitar influências indesejadas
colunas_numericas = dataset.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
dataset_escalado = scaler.fit_transform(dataset[colunas_numericas])

In [18]:
#Aplicando o KMeans sem o tratamento
#Definindo a função para encontrar o K ideal (Elbow Method)
def elbowmethod(dados):
    #Lista para preencher com a série de wcss
    wcss = []

    #Preenchendo a lista
    for k in range(1, 11):
        km = KMeans(n_clusters=k, random_state = 42)
        km.fit(dados)
        wcss.append(km.inertia_)
        
    #Criando o gráfico com a Plotly para demonstrar o resultado do Elbow Method
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=list(range(1, 11)),
        y=wcss,
        mode='lines+markers',
        name='WCSS',
        line=dict(color='royalblue', dash='dash'),
        marker=dict(size=8)
    ))
    
    fig.update_layout(
        title='Elbow Method',
        xaxis_title='Número de Clusters (k)',
        yaxis_title='WCSS',
        xaxis=dict(tickmode='linear', tick0=1, dtick=1),
        template='plotly_white'
    )
    
    return fig

In [19]:
#Aplicando o Elbow Method
elbow_method = elbowmethod(dataset_escalado)
elbow_method.show()

In [20]:
#Rodando o KMeans com o K ideal
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(dataset_escalado)
labels = kmeans.labels_
dataset['clusters_n'] = kmeans.labels_

In [21]:
#Checando os resultados
contagem = dataset['clusters_n'].value_counts().reset_index()
contagem.columns = ['Cluster', 'Frequência']

fig = px.bar(contagem, x='Cluster', y='Frequência', text='Frequência', title='Distribuição de Filmes por Cluster')

fig.show()

In [22]:
#Tratando os valores nulos numéricos
#Filtrando os registros por seus clusters categóricos
filtro_cluster0 = dataset[dataset['cluster'] == 0]
filtro_cluster1 = dataset[dataset['cluster'] == 1]
filtro_cluster2 = dataset[dataset['cluster'] == 2]
filtro_cluster3 = dataset[dataset['cluster'] == 3]
filtro_cluster4 = dataset[dataset['cluster'] == 4]
filtro_cluster5 = dataset[dataset['cluster'] == 5]

filtros_clusters = {'0': filtro_cluster0, '1': filtro_cluster1, '2': filtro_cluster2, '3': filtro_cluster3, '4': filtro_cluster4, '5': filtro_cluster5}

In [23]:
#Colunas numéricas a se pegar a mediana (com excessão de id)
print(colunas_numericas)

Index(['id', 'vote_average', 'vote_count', 'revenue', 'runtime', 'budget',
       'popularity'],
      dtype='object')


In [24]:
#Checando como essas colunas estão antes do tratamento
dataset[colunas_numericas].describe

<bound method NDFrame.describe of             id  vote_average  vote_count     revenue  runtime     budget  \
0        27205         8.364       34495   825532764      148  160000000   
1       157336         8.417       32571   701729206      169  165000000   
2          155         8.512       30619  1004558444      152  185000000   
3        19995         7.573       29815  2923706026      162  237000000   
4        24428         7.710       29166  1518815515      143  220000000   
...        ...           ...         ...         ...      ...        ...   
546185  776725         0.000           0           0       10          0   
546186  776726         0.000           0           0       13          0   
546187  776727         0.000           0           0        6          0   
546188  776730         0.000           0           0       11          0   
546189  776731         0.000           0           0       70          0   

        popularity  
0           83.952  
1          

In [25]:
#Definindo função para coletar as medianas de cada cluster para cada coluna
def coletar_medianas(filtros, dataset, colunas):
    dic_medianas = {}
    for cluster, dataframe in filtros.items():
        dic_medianas[cluster] = {}
        for coluna in colunas:
            if coluna == 'id':
                continue
            dic_medianas[cluster][coluna] = dataframe[coluna].median()
    return dic_medianas

In [26]:
#Coletando as medianas
medianas = coletar_medianas(filtros_clusters, dataset, colunas_numericas)
print(medianas)

{'0': {'vote_average': np.float64(0.0), 'vote_count': np.float64(0.0), 'revenue': np.float64(0.0), 'runtime': np.float64(53.0), 'budget': np.float64(0.0), 'popularity': np.float64(0.6)}, '1': {'vote_average': np.float64(4.0), 'vote_count': np.float64(1.0), 'revenue': np.float64(0.0), 'runtime': np.float64(85.0), 'budget': np.float64(0.0), 'popularity': np.float64(0.847)}, '2': {'vote_average': np.float64(4.0), 'vote_count': np.float64(1.0), 'revenue': np.float64(0.0), 'runtime': np.float64(75.0), 'budget': np.float64(0.0), 'popularity': np.float64(0.872)}, '3': {'vote_average': np.float64(5.0), 'vote_count': np.float64(3.0), 'revenue': np.float64(0.0), 'runtime': np.float64(80.0), 'budget': np.float64(0.0), 'popularity': np.float64(1.382)}, '4': {'vote_average': np.float64(0.0), 'vote_count': np.float64(0.0), 'revenue': np.float64(0.0), 'runtime': np.float64(13.0), 'budget': np.float64(0.0), 'popularity': np.float64(0.6)}, '5': {'vote_average': np.float64(4.104), 'vote_count': np.float

In [27]:
#Substituindo 0.0 pela mediana
for index, row in dataset.iterrows():
    for coluna in colunas_numericas:
        if coluna != 'id' and row[coluna] == 0:
            cluster_value = str(dataset.at[index, 'cluster'])

            dataset.at[index, coluna] = medianas[cluster_value][coluna]

In [28]:
#Checando como as colunas estão após o tratamento
dataset[colunas_numericas].describe

<bound method NDFrame.describe of             id  vote_average  vote_count     revenue  runtime     budget  \
0        27205         8.364       34495   825532764      148  160000000   
1       157336         8.417       32571   701729206      169  165000000   
2          155         8.512       30619  1004558444      152  185000000   
3        19995         7.573       29815  2923706026      162  237000000   
4        24428         7.710       29166  1518815515      143  220000000   
...        ...           ...         ...         ...      ...        ...   
546185  776725         5.000           3           0       10          0   
546186  776726         5.000           3           0       13          0   
546187  776727         0.000           0           0        6          0   
546188  776730         4.000           1           0       11          0   
546189  776731         0.000           0           0       70          0   

        popularity  
0           83.952  
1          

In [29]:
#Aplicando o Elbow Method pós tratamento
#Escalando o dataset
dataset_escalado = scaler.fit_transform(dataset[colunas_numericas])

In [30]:
#Checando se o K ideal mudou
elbow_method = elbowmethod(dataset_escalado)
elbow_method.show()

In [31]:
#Rodando o KMeans pós tratamento
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(dataset_escalado)
labels = kmeans.labels_
dataset['clusters_n'] = kmeans.labels_

In [32]:
#Checando os resultados
contagem = dataset['clusters_n'].value_counts().reset_index()
contagem.columns = ['Cluster', 'Frequência']

fig = px.bar(contagem, x='Cluster', y='Frequência', text='Frequência', title='Distribuição de Filmes por Cluster')

fig.show()

In [33]:
# Contando quantos registros pertencem a cada combinação de clusters
heatmap_data = dataset.groupby(['cluster', 'clusters_n']).size().reset_index(name='Contagem')

# Criando um heatmap para melhor observar a relação entre a clusterização categórica e a numérica
fig = px.density_heatmap(
    heatmap_data, 
    x='clusters_n', 
    y='cluster', 
    z='Contagem', 
    text_auto=True,  # Exibir valores no mapa
    color_continuous_scale='Viridis'
)

fig.show()